In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [14]:
import pandas as pd
df = pd.read_csv('/kaggle/input/titanic/train.csv', sep=',', header=0)
print(df.values)

[[1 0 3 ... 7.25 nan 'S']
 [2 1 1 ... 71.2833 'C85' 'C']
 [3 1 3 ... 7.925 nan 'S']
 ...
 [889 0 3 ... 23.45 nan 'S']
 [890 1 1 ... 30.0 'C148' 'C']
 [891 0 3 ... 7.75 nan 'Q']]


In [15]:
print(df.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [17]:
y_hat = pd.DataFrame(df['Survived'])

df_new = df.drop(columns=['Cabin', 'PassengerId', 'Survived'])

print(df_new)

     Pclass                                               Name     Sex   Age  \
0         3                            Braund, Mr. Owen Harris    male  22.0   
1         1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
2         3                             Heikkinen, Miss. Laina  female  26.0   
3         1       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0   
4         3                           Allen, Mr. William Henry    male  35.0   
..      ...                                                ...     ...   ...   
886       2                              Montvila, Rev. Juozas    male  27.0   
887       1                       Graham, Miss. Margaret Edith  female  19.0   
888       3           Johnston, Miss. Catherine Helen "Carrie"  female   NaN   
889       1                              Behr, Mr. Karl Howell    male  26.0   
890       3                                Dooley, Mr. Patrick    male  32.0   

     SibSp  Parch            Ticket    

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [63]:
#one hot encoding for sex and departure place

# need to label encode names and tickets + deal with NaN in Age 
# after that normalize all

df_train = pd.get_dummies(data=df_new, columns=['Sex', 'Embarked'], dtype=int)

#basic median seting (data is skewed left)
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())

print(df_train)

     Pclass                                               Name   Age  SibSp  \
0         3                            Braund, Mr. Owen Harris  22.0      1   
1         1  Cumings, Mrs. John Bradley (Florence Briggs Th...  38.0      1   
2         3                             Heikkinen, Miss. Laina  26.0      0   
3         1       Futrelle, Mrs. Jacques Heath (Lily May Peel)  35.0      1   
4         3                           Allen, Mr. William Henry  35.0      0   
..      ...                                                ...   ...    ...   
886       2                              Montvila, Rev. Juozas  27.0      0   
887       1                       Graham, Miss. Margaret Edith  19.0      0   
888       3           Johnston, Miss. Catherine Helen "Carrie"  28.0      1   
889       1                              Behr, Mr. Karl Howell  26.0      0   
890       3                                Dooley, Mr. Patrick  32.0      0   

     Parch            Ticket     Fare  Sex_female  

In [64]:
# age prediction
#df_age = pd.DataFrame(df[1])

# age_features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare']
# df_age_train = df.loc[df['Age'].notnull(), age_features + ['Age']]
# df_age_test = df.loc[df['Age'].isnull(), age_features]


In [65]:
df_train['Title'] = df_train['Name'].str.extract(r',\s*([^\.]+)\.')
print(df_train['Title'].unique())

['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'the Countess' 'Jonkheer']


In [66]:
#fixed it so less unique values of titles

df_train['Title'] = df_train['Title'].replace({'Mme': 'Mrs', 'Mlle': 'Ms', 'Miss': "Ms"})
rare_titles = ['Don', 'Rev', 'Dr', 'Major', 'Lady', 'Sir', 'Col', 'Capt', 'the Countess', 'Jonkheer']
df_train['Title'] = df_train['Title'].replace(rare_titles, 'Rare')

df_train = pd.get_dummies(df_train, columns=['Title'], dtype=int)

In [67]:
df_train['TicketHash'] = df_train['Ticket'].apply(lambda x: hash(x) % 10**6)

In [69]:
df_train = df_train.drop(columns=['Name', 'Ticket'])

In [70]:
print(df_train)

     Pclass   Age  SibSp  Parch     Fare  Sex_female  Sex_male  Embarked_C  \
0         3  22.0      1      0   7.2500           0         1           0   
1         1  38.0      1      0  71.2833           1         0           1   
2         3  26.0      0      0   7.9250           1         0           0   
3         1  35.0      1      0  53.1000           1         0           0   
4         3  35.0      0      0   8.0500           0         1           0   
..      ...   ...    ...    ...      ...         ...       ...         ...   
886       2  27.0      0      0  13.0000           0         1           0   
887       1  19.0      0      0  30.0000           1         0           0   
888       3  28.0      1      2  23.4500           1         0           0   
889       1  26.0      0      0  30.0000           0         1           1   
890       3  32.0      0      0   7.7500           0         1           0   

     Embarked_Q  Embarked_S  Title_Master  Title_Mr  Title_Mrs 

In [71]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_train[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'TicketHash']] = scaler.fit_transform(df_train[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'TicketHash']])

In [82]:
df_train = df_train.drop(columns=['TicketHash'])

df_train.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Mr,Title_Mrs,Title_Ms,Title_Rare
0,1.0,0.271174,0.125,0.0,0.014151,0,1,0,0,1,0,1,0,0,0
1,0.0,0.472229,0.125,0.0,0.139136,1,0,1,0,0,0,0,1,0,0
2,1.0,0.321438,0.000,0.0,0.015469,1,0,0,0,1,0,0,0,1,0
3,0.0,0.434531,0.125,0.0,0.103644,1,0,0,0,1,0,0,1,0,0
4,1.0,0.434531,0.000,0.0,0.015713,0,1,0,0,1,0,1,0,0,0


In [83]:
print(y_hat)

     Survived
0           0
1           1
2           1
3           1
4           0
..        ...
886         0
887         1
888         0
889         1
890         0

[891 rows x 1 columns]


In [84]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(df_train))

In [85]:
import tensorflow as tf

model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])

In [86]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ normalization_1 (Normalization)      │ (891, 15)                   │              31 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 31 (128.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 31 (128.00 B)

In [87]:
model.fit(df_train, y_hat, epochs=15, batch_size=2)

Epoch 1/15
446/446 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6795 - loss: 0.6007
Epoch 2/15
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8184 - loss: 0.4122
Epoch 3/15
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8309 - loss: 0.4105
Epoch 4/15
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8513 - loss: 0.3691
Epoch 5/15
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8416 - loss: 0.3871
Epoch 6/15
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8167 - loss: 0.4160
Epoch 7/15
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8591 - loss: 0.3674
Epoch 8/15
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8481 - loss: 0.3892
Epoch 9/15
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8634 - loss: 0.3507
Epoch 10/15
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8340 - loss: 0.3890
Epoch 11/15
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8499 - loss: 0.3784
Epoch 12/15
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step